In [1]:
% matplotlib inline

import numpy as np
import numpy.ma as ma
import pandas as pd
from math import *
import matplotlib.pyplot as plt
import scipy.io as sio
from mpl_toolkits.basemap import Basemap, cm
from netCDF4 import Dataset
import scipy.sparse as sp

import MITgcmutils as mit
from MITgcmutils import cs

/Users/sclayton/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
# script to interpolate ECCO2 output onto ECCO-Godae grid

rmwfile='/Users/sclayton/Documents/MIT_work/comparison/hrmp_e2_cs510_to_eg_v3_noland.nc'

# ECCO-Godae maskfile and number of vertical levels (needed to read mask).
# outhFacCfile='hFacC_eg.data';
outhFacCfile='/Volumes/My Passport/coarse_run/grid/HFacC.data'
nzout=23

# Load weights information
nc = Dataset(rmwfile, mode='r', mmap=False)


nlinks = len(nc.dimensions['num_links'])
ndin = len(nc.dimensions['src_grid_size'])
ndxout = nc.variables['dst_grid_dims'][0]
ndyout = nc.variables['dst_grid_dims'][1]
rmp_wgts = nc.variables['remap_matrix'][:,:]
rmp_src_i = nc.variables['src_address'][:]
rmp_dst_i = nc.variables['dst_address'][:]

nc.close()

hFacC = mit.rdmds('/Users/sclayton/Documents/darwin_model/e2_grid/hFacC')
hFacC = np.squeeze(hFacC[0,:,:])
outhfacc = np.fromfile(outhFacCfile, dtype = '>f').reshape(23, 160, 360)

dstmask1 = outhfacc[0,:,:]
dstmask1[dstmask1!=0]=1

# Create the weights matrix
nout,nin = 160*360,6*510*510
rm_mat = sp.coo_matrix((rmp_wgts[:,0], (rmp_dst_i[:]-1,rmp_src_i[:]-1)), shape=(nout,nin)).tocsr()


In [6]:
# interpolate upflux
# import HR data

monthly = np.zeros((4, 160, 360))

# import HR data
HRindir = '/Volumes/My Passport/high_res/'
in_file = '%supflux/WN.1999.data' % HRindir
HRWN = np.fromfile(in_file, dtype='>f').reshape(4, 510, 3060)

for v in range(4):
    phi = HRWN[v,:,:]
    phi[phi==0]=1e-30
    phi = phi.reshape(510, 3060)
    # Create a field with 1 over water and 0 over land
    phi_one = phi*0
    phi_one[phi!=0] = 1

    # Bin average the src field to the dst mesh
    foo = rm_mat*phi.reshape(510,6,510).transpose([1,0,2]).reshape(6*510*510)
    foo = foo.reshape(160,360)
    # Bin average the land-sea mask field to the dst mesh
    foo_one = rm_mat*phi_one.reshape(510,6,510).transpose([1,0,2]).reshape(6*510*510)
    foo_one = foo_one.reshape(160,360)
    a = foo_one[foo_one!=0]
    foo_one[foo_one!=0]=1./a

    # Zap points for which less than 1/alpha of the cell is wet.
    alpha = 20
    foo_one[foo_one>alpha] = 0.
    foo = foo*foo_one

    foo = foo *dstmask1
    foo[dstmask1==0] = 0.
        
    monthly[v,:,:] = foo

outfile='/Users/sclayton/Google Drive/work/comparison/data/annual/HR2CR/HR2CR_WN.1999.data'
monthly.astype('>f4').tofile(outfile)

In [3]:
# Regrid data

monthly=np.zeros((12,160,360))

for m in range(1, 13):

    infile='/Volumes/My Passport/high_res/Nuts/SurfNutsZoo.monthly.1999%02d.data' % m

    phi=np.fromfile(infile, dtype='>f')
    phi[phi==0]=1e-30
    phi = phi.reshape(21, 510, 3060)
    phi = phi[1,:,:]

    # Create a field with 1 over water and 0 over land
    phi_one = phi*0
    phi_one[phi!=0] = 1

    # Bin average the src field to the dst mesh
    foo = rm_mat*phi.reshape(510,6,510).transpose([1,0,2]).reshape(6*510*510)
    foo = foo.reshape(160,360)

    # Bin average the land-sea mask field to the dst mesh
    foo_one = rm_mat*phi_one.reshape(510,6,510).transpose([1,0,2]).reshape(6*510*510)
    foo_one = foo_one.reshape(160,360)
    a = foo_one[foo_one!=0]
    foo_one[foo_one!=0]=1./a

    # Zap points for which less than 1/alpha of the cell is wet.
    alpha = 20
    foo_one[foo_one>alpha] = 0.
    foo = foo*foo_one


    foo = foo *dstmask1
    foo[dstmask1==0] = 0.
    #fooall = foo
    #fooall[np.isnan(fooall)]==0 
    
    monthly[m-1,:,:] = foo

#  Write out the fooall field
# outfile='/Users/sclayton/Google Drive/work/comparison/data/monthly/HR2CR/HR2CR_N.1999.data'
# monthly.astype('>f4').tofile(outfile)
    